In [150]:
!pip install tensorflow

In [151]:
pip install tensorflow==2.16.1

Note: you may need to restart the kernel to use updated packages.


In [152]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [153]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

In [154]:
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [155]:
import tensorflow as tf

In [156]:
gpus = tf.config.experimental.list_logical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [157]:
for gpu in gpus:
  print(gpu)

In [158]:
len(gpus)

0

In [159]:
POS_PATH = "C:\\Users\\indug\\data\\data\\positive"
NEG_PATH = "C:\\Users\\indug\\data\\data\\negative"
ANC_PATH = "C:\\Users\\indug\\data\\data\\anchor"
mode = 0o666

In [160]:
os.makedirs(POS_PATH,mode, exist_ok = True)
os.makedirs(NEG_PATH,mode, exist_ok = True)
os.makedirs(ANC_PATH,mode, exist_ok = True)

In [161]:
# !tar -xf lfw-funneled.tgz

In [162]:
# import tarfile
# file_path = "D:\data\lfw-funneled.tgz"
# lfw = tarfile.open(file_path)
# lfw.extractall()
# lfw.close()

In [163]:
# for directory in os.listdir('lfw_funneled'):
#     for file in os.listdir(os.path.join('lfw_funneled', directory)):
#         EX_PATH = os.path.join('lfw_funneled', directory, file)
#         NEW_PATH = os.path.join(NEG_PATH, file)
#         os.replace(EX_PATH, NEW_PATH)

In [164]:
# os. getcwd()
os.listdir('lfw_funneled')

['Aaron_Eckhart',
 'Aaron_Guiel',
 'Aaron_Patterson',
 'Aaron_Peirsol',
 'Aaron_Pena',
 'Aaron_Sorkin',
 'Aaron_Tippin',
 'Abbas_Kiarostami',
 'Abba_Eban',
 'Abdel_Aziz_Al-Hakim',
 'Abdel_Madi_Shabneh',
 'Abdel_Nasser_Assidi',
 'Abdoulaye_Wade',
 'Abdulaziz_Kamilov',
 'Abdullah',
 'Abdullah_Ahmad_Badawi',
 'Abdullah_al-Attiyah',
 'Abdullah_Gul',
 'Abdullah_Nasseef',
 'Abdullatif_Sener',
 'Abdul_Majeed_Shobokshi',
 'Abdul_Rahman',
 'Abel_Aguilar',
 'Abel_Pacheco',
 'Abid_Hamid_Mahmud_Al-Tikriti',
 'Abner_Martinez',
 'Abraham_Foxman',
 'Aby_Har-Even',
 'Adam_Ant',
 'Adam_Freier',
 'Adam_Herbert',
 'Adam_Kennedy',
 'Adam_Mair',
 'Adam_Rich',
 'Adam_Sandler',
 'Adam_Scott',
 'Adelina_Avila',
 'Adel_Al-Jubeir',
 'Adisai_Bodharamik',
 'Adolfo_Aguilar_Zinser',
 'Adolfo_Rodriguez_Saa',
 'Adoor_Gopalakarishnan',
 'Adriana_Lima',
 'Adriana_Perez_Navarro',
 'Adrianna_Zuzic',
 'Adrian_Annus',
 'Adrian_Fernandez',
 'Adrian_McPherson',
 'Adrian_Murrell',
 'Adrian_Nastase',
 'Adrien_Brody',
 'Afton_S

In [165]:
import uuid

In [166]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

'C:\\Users\\indug\\data\\data\\anchor\\df7da63f-0bd7-11ef-a6cd-a0e70ba2ec01.jpg'

In [167]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [168]:
plt.imshow(frame)
frame.shape

NameError: name 'frame' is not defined

In [ ]:
# plt.imshow(frame[120:120+250,200:200+250, :])

In [169]:
def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        # img = tf.image.stateless_random_crop(img, size=(20,20,3), seed=(1,2))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))
            
        data.append(img)
    
    return data

In [170]:

import os
import uuid

In [ ]:
# img_path = os.path.join(ANC_PATH, '924e839c-135f-11ec-b54e-a0cec8d2d278.jpg')
# img = cv2.imread(img_path)
# augmented_images = data_aug(img)

# for image in augmented_images:
#     cv2.imwrite(os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [ ]:
for file_name in os.listdir(os.path.join(ANC_PATH)):
    img_path = os.path.join(ANC_PATH, file_name)
    img = cv2.imread(img_path)
    augmented_images = data_aug(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())), image.numpy())

In [171]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(800)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(800)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(800)

In [172]:

dir_test = anchor.as_numpy_iterator()

In [173]:
print(dir_test.next())

b'C:\\Users\\indug\\data\\data\\anchor\\4e73e373-00cb-11ef-8662-a0e70ba2ec01.jpg'


In [174]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (200,200))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [175]:
img = preprocess('\\Users\\indug\\data\\data\\anchor\\3c3af058-f10c-11ee-a1cd-d7d88491879d.jpg')

In [1]:
# plt.imshow(img)

In [177]:
img.numpy().max()

1.0

In [178]:
# dataset.map(preprocess)

In [179]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [180]:
samples = data.as_numpy_iterator()
example = samples.next()
example
# tf.zeros(len(negative))

(b'C:\\Users\\indug\\data\\data\\anchor\\b07f3c39-00cb-11ef-8cdb-a0e70ba2ec01.jpg',
 b'C:\\Users\\indug\\data\\data\\positive\\cdab8469-00c9-11ef-9af5-a0e70ba2ec01.jpg',
 1.0)

In [181]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [182]:
res = preprocess_twin(*example)
len(res)

3

In [2]:
# plt.imshow(res[1])

In [184]:
res[2]

1.0

In [185]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=100000)

In [186]:
samples = data.as_numpy_iterator()

In [187]:
samp = samples.next()

In [3]:
# plt.imshow(samp[0])

In [4]:
# plt.imshow(samp[1])

In [190]:
samp[2]

1.0

In [191]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [192]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 200, 200, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 200, 200, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [193]:

# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [194]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 200, 200, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 200, 200, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [195]:
inp = tf.keras.layers.Input(shape=(200, 200, 3), name='input_image')

c1 = Conv2D(64, (10,10), activation='relu')(inp)
m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

c2 = Conv2D(128, (7,7), activation='relu')(m1)
m2 = MaxPooling2D(64, (2,2), padding='same')(c2)

c3 = Conv2D(128, (4,4), activation='relu')(m2)
m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

c4 = Conv2D(256, (4,4), activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation='sigmoid')(f1)
mod = Model(inputs=[inp], outputs=[d1], name='embedding')

mod.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)             │ (None, 200, 200, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 191, 191, 64)        │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 96, 96, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 90, 90, 128)         │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 45, 45, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 42, 42, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 21, 21, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_19 (Conv2D)                   │ (None, 18, 18, 256)         │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 82944)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 4096)                │     339,742,720 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 340,950,336 (1.27 GB)

 Trainable params: 340,950,336 (1.27 GB)

 Non-trainable params: 0 (0.00 B)

In [196]:
def make_embedding(): 
    inp = tf.keras.layers.Input(shape=(200, 200, 3), name='input_image')
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [197]:
embedding = make_embedding()

embedding.summary()

Model: "embedding"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)             │ (None, 200, 200, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_20 (Conv2D)                   │ (None, 191, 191, 64)        │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 96, 96, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 90, 90, 128)         │         401,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 45, 45, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 42, 42, 128)         │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 21, 21, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 18, 18, 256)         │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 82944)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 4096)                │     339,742,720 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 340,950,336 (1.27 GB)

 Trainable params: 340,950,336 (1.27 GB)

 Non-trainable params: 0 (0.00 B)

#building distance layer

In [198]:
# Siamese L1 Distance class
class L1Dist(tf.keras.layers.Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        input_embedding = tf.convert_to_tensor(input_embedding)
#         validation_embedding = tf.convert_to_tensor(validation_embedding)
        return tf.math.abs(input_embedding - validation_embedding)

In [199]:
l1 = L1Dist()

In [200]:
# l1(anchor_embedding, validation_embedding)

In [201]:
input_image = tf.keras.layers.Input(name='input_img', shape=(200,200,3))
validation_image = tf.keras.layers.Input(name='validation_img', shape=(200,200,3))

In [202]:
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)

In [203]:
siamese_layer = L1Dist()

In [204]:
 distances = siamese_layer(embedding(input_image), embedding(validation_image))

In [205]:
classifier = Dense(1, activation='sigmoid')(distances)

In [206]:
classifier

<KerasTensor shape=(1, None, 1), dtype=float32, sparse=False, name=keras_tensor_79>

In [207]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = tf.keras.layers.Input(name='input_img', shape=(200,200,3))
    
    # Validation image in the network 
    validation_image = tf.keras.layers.Input(name='validation_img', shape=(200,200,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs= classifier, name='SiameseNetwork')

In [208]:
Siamese_Network =  Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [209]:
siamese_model = make_siamese_model()
Siamese_Network.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_img (InputLayer)        │ (None, 200, 200, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ validation_img (InputLayer)   │ (None, 200, 200, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Functional)        │ (None, 4096)              │     340,950,336 │ input_img[0][0],           │
│                               │                           │                 │ validation_img[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ l1_dist_7 (L1Dist)            │ (1, None, 4096)           │               0 │ embedding[2][0],           │
│                               │                           │                 │ embedding[3][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_10 (Dense)              │ (1, None, 1)              │           4,097 │ l1_dist_7[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 340,954,433 (1.27 GB)

 Trainable params: 340,954,433 (1.27 GB)

 Non-trainable params: 0 (0.00 B)

In [210]:
#training
#setup loss and optimizer

In [211]:

binary_cross_loss = tf.losses.BinaryCrossentropy()

In [212]:
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

In [213]:

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [214]:
test_batch = train_data.as_numpy_iterator()

In [215]:

batch_1 = test_batch.next()
len(batch_1[2])

16

In [216]:
X = batch_1[:2]
np.array(X).shape

(2, 16, 200, 200, 3)

In [217]:
y = batch_1[2]
np.array(y).shape


(16,)

In [218]:
# # tf.losses.BinaryCrossentropy??

In [219]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Flatten yhat to match the shape of y
        yhat = tf.squeeze(yhat, axis=-1)
        
        # Reshape y to match the shape of yhat
        y = tf.reshape(y, yhat.shape)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    # Return loss
    return loss

In [220]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [221]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [222]:
EPOCHS = 12

In [223]:
train(train_data, EPOCHS)


 Epoch 1/12
Tensor("binary_crossentropy/truediv:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/truediv:0", shape=(), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/steps/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step/ste
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step/ste
1/1 ━━━━━━━━━━━━━━━━━━

In [224]:

# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [225]:
# Get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [226]:
y_hat = siamese_model.predict([test_input, test_val])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [227]:
# [1 if prediction > 0.5 else 0 for prediction in y_hat ]

In [228]:
y_true


array([1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1.],
      dtype=float32)

In [229]:

# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

1.0

In [230]:

# Creating a metric object 
m = Precision()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

1.0

In [231]:

r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 

print(r.result().numpy(), p.result().numpy())

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 

In [5]:
# # Set plot size 
# plt.figure(figsize=(10,8))

# # Set first subplot
# plt.subplot(1,2,1)
# plt.imshow(test_input[0])

# # Set second subplot
# plt.subplot(1,2,2)
# plt.imshow(test_val[0])

# # Renders cleanly
# plt.show()

In [233]:
# # Save weights
# siamese_model.save('siamesemodelv2.h5')


In [235]:

L1Dist

__main__.L1Dist

In [236]:
# siamese_model = tf.keras.models.load_model('siamesemodelv2.h5', 
#                                    custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [237]:
siamese_model.predict([test_input, test_val])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[[3.9274517e-05],
        [1.3413322e-04],
        [9.9999940e-01],
        [9.9778003e-01],
        [2.0209591e-04],
        [9.9900085e-01],
        [9.9997973e-01],
        [7.9581910e-04],
        [1.1415346e-02],
        [9.9999976e-01],
        [1.3335226e-05],
        [9.9999303e-01],
        [1.0676109e-03],
        [9.9955076e-01],
        [9.9992496e-01],
        [9.9999946e-01]]], dtype=float32)

In [238]:
siamese_model.summary()

Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_img (InputLayer)        │ (None, 200, 200, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ validation_img (InputLayer)   │ (None, 200, 200, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Functional)        │ (None, 4096)              │     340,950,336 │ input_img[0][0],           │
│                               │                           │                 │ validation_img[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ l1_dist_8 (L1Dist)            │ (1, None, 4096)           │               0 │ embedding[4][0],           │
│                               │                           │                 │ embedding[5][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_11 (Dense)              │ (1, None, 1)              │           4,097 │ l1_dist_8[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 340,954,433 (1.27 GB)

 Trainable params: 340,954,433 (1.27 GB)

 Non-trainable params: 0 (0.00 B)

In [239]:

# application_data\verification_images

In [240]:

os.listdir(os.path.join('application_data', 'verification_images'))

['2818399c-f10c-11ee-a38b-d7d88491879d.jpg',
 '28502cbb-f10c-11ee-9816-d7d88491879d.jpg',
 '387e0f08-fbec-11ee-976b-d7d88491879d.jpg',
 '42149357-f10c-11ee-a160-d7d88491879d.jpg',
 '4237e5e7-f10c-11ee-83e5-d7d88491879d.jpg',
 '4281f37e-f10c-11ee-bafc-d7d88491879d.jpg',
 '453e1dad-fbec-11ee-b846-d7d88491879d.jpg',
 '4585ff70-fbec-11ee-9fc6-d7d88491879d.jpg',
 '480f99d5-fbec-11ee-912f-d7d88491879d.jpg',
 '482d1806-fbec-11ee-93c9-d7d88491879d.jpg',
 '5508a9dd-f10c-11ee-84a6-d7d88491879d.jpg',
 '557b8707-f10c-11ee-bcd0-d7d88491879d.jpg',
 '5594aa9b-f10c-11ee-9948-d7d88491879d.jpg',
 '56fab950-f10c-11ee-bb63-d7d88491879d.jpg',
 '57148a19-f10c-11ee-8862-d7d88491879d.jpg',
 '57e7a027-f10c-11ee-9d12-d7d88491879d.jpg',
 '5808ab5a-f10c-11ee-acbb-d7d88491879d.jpg',
 '596a3953-f10c-11ee-bd96-d7d88491879d.jpg',
 '59a40ab3-f10c-11ee-9469-d7d88491879d.jpg',
 '59c0c1ea-f10c-11ee-b3b3-d7d88491879d.jpg']

In [241]:
os.path.join('application_data', 'input_image', 'input_image.jpg')

'application_data\\input_image\\input_image.jpg'

In [242]:
os.path.join('application_data', 'input_image', 'input_image.jpg')

'application_data\\input_image\\input_image.jpg'

In [243]:
for image in os.listdir(os.path.join('application_data', 'verification_images')):
    validation_img = os.path.join('application_data', 'verification_images', image)
    print(validation_img)


application_data\verification_images\2818399c-f10c-11ee-a38b-d7d88491879d.jpg
application_data\verification_images\28502cbb-f10c-11ee-9816-d7d88491879d.jpg
application_data\verification_images\387e0f08-fbec-11ee-976b-d7d88491879d.jpg
application_data\verification_images\42149357-f10c-11ee-a160-d7d88491879d.jpg
application_data\verification_images\4237e5e7-f10c-11ee-83e5-d7d88491879d.jpg
application_data\verification_images\4281f37e-f10c-11ee-bafc-d7d88491879d.jpg
application_data\verification_images\453e1dad-fbec-11ee-b846-d7d88491879d.jpg
application_data\verification_images\4585ff70-fbec-11ee-9fc6-d7d88491879d.jpg
application_data\verification_images\480f99d5-fbec-11ee-912f-d7d88491879d.jpg
application_data\verification_images\482d1806-fbec-11ee-93c9-d7d88491879d.jpg
application_data\verification_images\5508a9dd-f10c-11ee-84a6-d7d88491879d.jpg
application_data\verification_images\557b8707-f10c-11ee-bcd0-d7d88491879d.jpg
application_data\verification_images\5594aa9b-f10c-11ee-9948-d7d

In [244]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [256]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
#         hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#         h, s, v = cv2.split(hsv)

#         lim = 255 - 10
#         v[v > lim] = 255
#         v[v <= lim] -= 10
        
#         final_hsv = cv2.merge((h, s, v))
#         img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(siamese_model, 0.4, 0.4)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
False
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━

In [250]:
np.sum(np.squeeze(results) > 0.9)

20

In [248]:
results


[array([[[0.00653418]]], dtype=float32),
 array([[[0.0058801]]], dtype=float32),
 array([[[0.9932545]]], dtype=float32),
 array([[[0.00598905]]], dtype=float32),
 array([[[0.00664159]]], dtype=float32),
 array([[[0.00565366]]], dtype=float32),
 array([[[0.99999106]]], dtype=float32),
 array([[[0.999981]]], dtype=float32),
 array([[[0.99988776]]], dtype=float32),
 array([[[0.9997016]]], dtype=float32),
 array([[[0.00209531]]], dtype=float32),
 array([[[0.00229803]]], dtype=float32),
 array([[[0.00105023]]], dtype=float32),
 array([[[0.00278525]]], dtype=float32),
 array([[[0.00298091]]], dtype=float32),
 array([[[0.00364007]]], dtype=float32),
 array([[[0.00390062]]], dtype=float32),
 array([[[0.00428302]]], dtype=float32),
 array([[[0.00323755]]], dtype=float32),
 array([[[0.00373747]]], dtype=float32)]

In [5]:
help()

Welcome to Python 3.11's help utility! If this is your first time using
Python, you should definitely check out the tutorial at
https://docs.python.org/3.11/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To get a list of available
modules, keywords, symbols, or topics, enter "modules", "keywords",
"symbols", or "topics".

Each module also comes with a one-line summary of what it does; to list
the modules whose name or summary contain a given string such as "spam",
enter "modules spam".

To quit this help utility and return to the interpreter,
enter "q" or "quit".



help>  keras.layers


Help on package keras.layers in keras:

NAME
    keras.layers - DO NOT EDIT.

DESCRIPTION
    This file was autogenerated. Do not edit it by hand,
    since your modifications would be overwritten.

PACKAGE CONTENTS


FILE
    c:\users\indug\anaconda3\lib\site-packages\keras\layers\__init__.py




help>  keras


Help on package keras:

NAME
    keras - DO NOT EDIT.

DESCRIPTION
    This file was autogenerated. Do not edit it by hand,
    since your modifications would be overwritten.

PACKAGE CONTENTS
    _tf_keras (package)
    activations (package)
    applications (package)
    backend (package)
    callbacks (package)
    config (package)
    constraints (package)
    datasets (package)
    distribution (package)
    dtype_policies (package)
    export (package)
    initializers (package)
    layers (package)
    legacy (package)
    losses (package)
    metrics (package)
    mixed_precision (package)
    models (package)
    ops (package)
    optimizers (package)
    preprocessing (package)
    quantizers (package)
    random (package)
    regularizers (package)
    saving (package)
    src (package)
    tree (package)
    utils (package)

VERSION
    3.2.1

FILE
    c:\users\indug\anaconda3\lib\site-packages\keras\__init__.py




help>  keras.models


Help on package keras.models in keras:

NAME
    keras.models - DO NOT EDIT.

DESCRIPTION
    This file was autogenerated. Do not edit it by hand,
    since your modifications would be overwritten.

PACKAGE CONTENTS


FILE
    c:\users\indug\anaconda3\lib\site-packages\keras\models\__init__.py



You are now leaving help and returning to the Python interpreter.
If you want to ask for help on a particular object directly from the
interpreter, you can type "help(object)".  Executing "help('string')"
has the same effect as typing a particular string at the help> prompt.


In [3]:
sklearn.metrics??

Object `sklearn.metrics` not found.


In [6]:
pip install keras.models

   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.1 MB 2.7 MB/s eta 0:00:05
   ---------------------------------------- 0.1/12.1 MB 1.6 MB/s eta 0:00:08
    --------------------------------------- 0.2/12.1 MB 1.7 MB/s eta 0:00:08
    --------------------------------------- 0.3/12.1 MB 1.5 MB/s eta 0:00:08
   - -------------------------------------- 0.4/12.1 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.5/12.1 MB 1.8 MB/s eta 0:00:07
   - -------------------------------------- 0.6/12.1 MB 1.8 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/12.1 MB 1.8 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/12.1 MB 1.9 MB/s eta 0:00:07
   -- ------------------------------------- 0.9/12.1 MB 1.9 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/12.1 MB 1.8 MB/s eta 0:00:07
   --- ------------------------------------ 1.1/12.1 MB 1.9 MB/s eta 0:00:06
   ---

In [11]:
help()

Welcome to Python 3.11's help utility! If this is your first time using
Python, you should definitely check out the tutorial at
https://docs.python.org/3.11/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To get a list of available
modules, keywords, symbols, or topics, enter "modules", "keywords",
"symbols", or "topics".

Each module also comes with a one-line summary of what it does; to list
the modules whose name or summary contain a given string such as "spam",
enter "modules spam".

To quit this help utility and return to the interpreter,
enter "q" or "quit".



help>  sklearn.model_selection.LinearRegression


No Python documentation found for 'sklearn.model_selection.LinearRegression'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.



help>  sklearn.linear_model.LinearRegression


Help on class LinearRegression in sklearn.linear_model:

sklearn.linear_model.LinearRegression = class LinearRegression(sklearn.base.MultiOutputMixin, sklearn.base.RegressorMixin, LinearModel)
 |  sklearn.linear_model.LinearRegression(*, fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
 |  
 |  Ordinary least squares Linear Regression.
 |  
 |  LinearRegression fits a linear model with coefficients w = (w1, ..., wp)
 |  to minimize the residual sum of squares between the observed targets in
 |  the dataset, and the targets predicted by the linear approximation.
 |  
 |  Parameters
 |  ----------
 |  fit_intercept : bool, default=True
 |      Whether to calculate the intercept for this model. If set
 |      to False, no intercept will be used in calculations
 |      (i.e. data is expected to be centered).
 |  
 |  copy_X : bool, default=True
 |      If True, X will be copied; else, it may be overwritten.
 |  
 |  n_jobs : int, default=None
 |      The number of jobs to use 